In [1]:
import numpy as np

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def sigmoid_output_to_derivative(output):
    return output*(1-output)

In [2]:
# 训练数据生成
int2binary = {}
binary_dim = 8

largest_number = pow(2, binary_dim)
binary = np.unpackbits(
    np.array([range(largest_number)], dtype=np.uint8).T, axis=1)
for i in range(largest_number):
    int2binary[i] = binary[i]

# 初始化一些变量
alpha = 0.1 #学习率
input_dim = 2   #输入的大小
hidden_dim = 8  #隐含层的大小
output_dim = 1  #输出层的大小

# 随机初始化权重
synapse_0 = 2 * np.random.random((hidden_dim, input_dim)) - 1   #(8, 2)
synapse_1 = 2 * np.random.random((output_dim, hidden_dim)) - 1  #(1, 8)
synapse_h = 2 * np.random.random((hidden_dim, hidden_dim)) - 1  #(8, 8)

synapse_0_update = np.zeros_like(synapse_0) #(8, 2)
synapse_1_update = np.zeros_like(synapse_1) #(1, 8)
synapse_h_update = np.zeros_like(synapse_h) #(8, 8)
for j in range(10000):
# 计算一个简单的加法(a + b = c)
    a_int = np.random.randint(largest_number / 2)  # int version
    a = int2binary[a_int]  # binary encoding

    b_int = np.random.randint(largest_number / 2)  # int version
    b = int2binary[b_int]  # binary encoding

    # 正确答案
    c_int = a_int + b_int
    c = int2binary[c_int]

    # 待存放预测值
    d = np.zeros_like(c)

    overallError = 0

    layer_2_deltas = list()  # 输出层误差
    layer_1_values = list()  # 第一层的值（隐含状态）
    # 第一个隐含状态需要0作为它的上一个隐含状态
    layer_1_values.append(np.zeros(hidden_dim))  
    # 前向传播：在二进制编码中沿着位置移动
    for position in range(binary_dim):
        # 生成输入和输出
        X = np.array([[a[binary_dim - position - 1], b[binary_dim - position - 1]]])
        y = np.array([[c[binary_dim - position - 1]]]).T

        # hidden layer (input ~+ prev_hidden)
#         layer_1 = sigmoid(np.dot(X, synapse_0) + np.dot(layer_1_values[-1], synapse_h))
        layer_1 = sigmoid(np.dot(X,synapse_0) + np.dot(layer_1_values[-1],synapse_h))

        # output layer (new binary representation)
        layer_2 = sigmoid(np.dot(layer_1, synapse_1))

        # 计算预测误差（预测值与真实值的差）
        layer_2_error = y - layer_2
        layer_2_deltas.append((layer_2_error) * sigmoid_output_to_derivative(layer_2))
        overallError += np.abs(layer_2_error[0])
        d[binary_dim - position - 1] = np.round(layer_2[0][0])

        # 拷贝layer_1的值，下次使用
        layer_1_values.append(copy.deepcopy(layer_1))

    future_layer_1_delta = np.zeros(hidden_dim)
    for position in range(binary_dim):
        X = np.array([[a[position], b[position]]])
        layer_1 = layer_1_values[-position - 1]
        prev_layer_1 = layer_1_values[-position - 2]

        # 从列表中取出当前输出层的误差
        layer_2_delta = layer_2_deltas[-position - 1]
        # 计算当前隐含层的误差
        layer_1_delta = (future_layer_1_delta.dot(synapse_h.T) + \
                         layer_2_delta.dot(synapse_1.T)) * sigmoid_output_to_derivative(layer_1)
        # 生成权值更新的量（但是还没真正的更新权值）
        synapse_1_update += np.atleast_2d(layer_1).T.dot(layer_2_delta)
        synapse_h_update += np.atleast_2d(prev_layer_1).T.dot(layer_1_delta)
        synapse_0_update += X.T.dot(layer_1_delta)

        future_layer_1_delta = layer_1_delta
    # 完成反向传播，更新权值
    synapse_0 += synapse_0_update * alpha
    synapse_1 += synapse_1_update * alpha
    synapse_h += synapse_h_update * alpha
    # 别忘了重置update变量
    synapse_0_update *= 0
    synapse_1_update *= 0
    synapse_h_update *= 0
    if (j % 1000 == 0):
        print("Error:" + str(overallError))
        print("Pred:" + str(d))
        print("True:" + str(c))
        out = 0
        for index, x in enumerate(reversed(d)):
            out += x * pow(2, index)
        print(str(a_int) + " + " + str(b_int) + " = " + str(out))
        print("------------")

ValueError: shapes (1,2) and (8,2) not aligned: 2 (dim 1) != 8 (dim 0)